Reference https://machinelearningmastery.com/how-to-develop-a-cnn-from-scratch-for-cifar-10-photo-classification/

In [1]:
import keras
from keras.datasets import cifar10
from keras.models import Model, Sequential
from keras.layers import Dense, Dropout, Flatten, Input, AveragePooling2D, Activation
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras.layers import Concatenate
from tensorflow.keras.optimizers import SGD
from keras import regularizers

In [2]:
batch_size = 128
num_classes = 10
epochs = 250
l = 12
num_filter = 36 
compression = 0.5 
dropout_rate = 0.2

In [3]:
(X_train, y_train), (X_test, y_test) = cifar10.load_data()
img_height, img_width, channel = X_train.shape[1],X_train.shape[2],X_train.shape[3]

y_train = keras.utils.np_utils.to_categorical(y_train, num_classes)
y_test = keras.utils.np_utils.to_categorical(y_test, num_classes)

170508288/170498071 [==============================] - 2s 0us/step


In [4]:
def denseblock(input, num_filter = 12, dropout_rate = 0.2):
  global compression
  temp = input
  for _ in range(l):
      BatchNorm = BatchNormalization()(temp)
      relu = Activation('relu')(BatchNorm)
      Conv2D_3_3 = Conv2D(int(num_filter*compression), (3,3), use_bias=False ,padding='same')(relu)
      concat = Concatenate(axis=-1)([temp,Conv2D_3_3])        
      temp = concat        
  return temp

def transition(input, num_filter = 12, dropout_rate = 0.2):
  global compression
  BatchNorm = BatchNormalization()(input)
  relu = Activation('relu')(BatchNorm)
  Conv2D_BottleNeck = Conv2D(int(num_filter*compression), (1,1), use_bias=False, kernel_regularizer = regularizers.l1() ,padding='same')(relu)
  avg = AveragePooling2D(pool_size=(2,2))(Conv2D_BottleNeck)  
  return avg

def output_layer(input):
  global compression
  BatchNorm = BatchNormalization()(input)
  relu = Activation('relu')(BatchNorm)
  AvgPooling = AveragePooling2D(pool_size=(2,2))(relu)
  temp = Conv2D(num_classes, kernel_size = (2,2))(AvgPooling)
  output = Activation('softmax')(temp)
  flat = Flatten()(output)
  return flat

In [5]:
num_filter = 36
dropout_rate = 0.2
l= 12
input = Input(shape=(img_height, img_width, channel,))
First_Conv2D = Conv2D(num_filter, (3,3), use_bias=False ,padding='same')(input)

First_Block = denseblock(First_Conv2D, num_filter, dropout_rate)
First_Transition = transition(First_Block, num_filter, dropout_rate)

Second_Block = denseblock(First_Transition, num_filter, dropout_rate)
Second_Transition = transition(Second_Block, num_filter, dropout_rate)

Third_Block = denseblock(Second_Transition, num_filter, dropout_rate)
Third_Transition = transition(Third_Block, num_filter, dropout_rate)

Last_Block = denseblock(Third_Transition,  num_filter, dropout_rate)
output = output_layer(Last_Block)

In [6]:
model = Model(inputs=[input], outputs=[output])
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 32, 32, 3)]  0           []                               
                                                                                                  
 conv2d (Conv2D)                (None, 32, 32, 36)   972         ['input_1[0][0]']                
                                                                                                  
 batch_normalization (BatchNorm  (None, 32, 32, 36)  144         ['conv2d[0][0]']                 
 alization)                                                                                       
                                                                                                  
 activation (Activation)        (None, 32, 32, 36)   0           ['batch_normalization[0][0]']

In [7]:
from keras.preprocessing.image import ImageDataGenerator
datagen = ImageDataGenerator(rotation_range = 15, horizontal_flip = True, width_shift_range = 0.1, height_shift_range = 0.1, zoom_range = 0.2, shear_range = 15)
datagen.fit(X_train)

In [8]:
from keras.callbacks import ModelCheckpoint
ckpt = ModelCheckpoint(filepath = 'model.hdf5', monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')

In [9]:
model.compile(loss='categorical_crossentropy',optimizer=SGD(0.01, momentum = 0.7),metrics=['accuracy'])

In [ ]:
model.fit(datagen.flow(X_train, y_train, batch_size), steps_per_epoch = X_train.shape[0]/batch_size, epochs = 30, validation_data =(X_test, y_test), callbacks = [ckpt])
model.save_weights('30epochs.h5')

Epoch 1/30
390/390 [==============================] - 121s 260ms/step - loss: 5.6446 - accuracy: 0.3960 - val_loss: 2.7081 - val_accuracy: 0.2483
Epoch 2/30
390/390 [==============================] - 98s 250ms/step - loss: 1.8941 - accuracy: 0.4644 - val_loss: 2.2132 - val_accuracy: 0.3664
Epoch 3/30
390/390 [==============================] - 98s 251ms/step - loss: 1.6627 - accuracy: 0.5283 - val_loss: 2.0398 - val_accuracy: 0.4091
Epoch 4/30
390/390 [==============================] - 98s 251ms/step - loss: 1.5333 - accuracy: 0.5653 - val_loss: 1.7086 - val_accuracy: 0.5257
Epoch 5/30
390/390 [==============================] - 98s 251ms/step - loss: 1.4435 - accuracy: 0.5940 - val_loss: 1.5918 - val_accuracy: 0.5513
Epoch 6/30
390/390 [==============================] - 98s 250ms/step - loss: 1.3706 - accuracy: 0.6207 - val_loss: 1.4612 - val_accuracy: 0.5976
Epoch 7/30
390/390 [==============================] - 98s 251ms/step - loss: 1.3094 - accuracy: 0.6412 - val_loss: 1.8103 - val_a

In [ ]:
model.fit(datagen.flow(X_train, y_train, batch_size), steps_per_epoch = X_train.shape[0]/batch_size, epochs = 30, validation_data =(X_test, y_test), callbacks = [ckpt])
model.save_weights('60epochs.h5')

Epoch 1/30
391/390 [==============================] - ETA: 0s - loss: 0.8250 - accuracy: 0.8095
Epoch 1: val_accuracy improved from -inf to 0.81060, saving model to model.hdf5
390/390 [==============================] - 98s 250ms/step - loss: 0.8250 - accuracy: 0.8095 - val_loss: 0.8335 - val_accuracy: 0.8106
Epoch 2/30
391/390 [==============================] - ETA: 0s - loss: 0.8168 - accuracy: 0.8118
Epoch 2: val_accuracy did not improve from 0.81060
390/390 [==============================] - 97s 249ms/step - loss: 0.8168 - accuracy: 0.8118 - val_loss: 0.9283 - val_accuracy: 0.7832
Epoch 3/30
391/390 [==============================] - ETA: 0s - loss: 0.8019 - accuracy: 0.8168
Epoch 3: val_accuracy did not improve from 0.81060
390/390 [==============================] - 97s 249ms/step - loss: 0.8019 - accuracy: 0.8168 - val_loss: 0.9795 - val_accuracy: 0.7702
Epoch 4/30
391/390 [==============================] - ETA: 0s - loss: 0.8023 - accuracy: 0.8188
Epoch 4: val_accuracy improved f

In [ ]:
from keras.models import load_model
model = load_model('model.hdf5')
from keras.preprocessing.image import ImageDataGenerator
datagen = ImageDataGenerator(rotation_range = 15, horizontal_flip = True, width_shift_range = 0.1, height_shift_range = 0.1, zoom_range = 0.2, shear_range = 15)
datagen.fit(X_train)
model.fit(datagen.flow(X_train, y_train, batch_size), steps_per_epoch = X_train.shape[0]/batch_size, epochs = 30, validation_data =(X_test, y_test), callbacks = [ckpt])
model.save_weights('72epochs.h5')

Epoch 1/30
391/390 [==============================] - ETA: 0s - loss: 0.6849 - accuracy: 0.8527
Epoch 1: val_accuracy improved from 0.83360 to 0.84560, saving model to model.hdf5
390/390 [==============================] - 99s 247ms/step - loss: 0.6849 - accuracy: 0.8527 - val_loss: 0.7226 - val_accuracy: 0.8456
Epoch 2/30
391/390 [==============================] - ETA: 0s - loss: 0.6782 - accuracy: 0.8548
Epoch 2: val_accuracy did not improve from 0.84560
390/390 [==============================] - 98s 250ms/step - loss: 0.6782 - accuracy: 0.8548 - val_loss: 0.7559 - val_accuracy: 0.8342
Epoch 3/30
391/390 [==============================] - ETA: 0s - loss: 0.6765 - accuracy: 0.8552
Epoch 3: val_accuracy did not improve from 0.84560
390/390 [==============================] - 98s 250ms/step - loss: 0.6765 - accuracy: 0.8552 - val_loss: 0.7650 - val_accuracy: 0.8318
Epoch 4/30
391/390 [==============================] - ETA: 0s - loss: 0.6662 - accuracy: 0.8589
Epoch 4: val_accuracy did not

In [ ]:
model.load_weights('72epochs.h5')
from keras.preprocessing.image import ImageDataGenerator
datagen = ImageDataGenerator(rotation_range = 15, horizontal_flip = True, width_shift_range = 0.1, height_shift_range = 0.1, zoom_range = 0.2, shear_range = 15)
datagen.fit(X_train)
keras.backend.set_value(model.optimizer.lr, .001)
model.fit(datagen.flow(X_train, y_train, batch_size), steps_per_epoch = X_train.shape[0]/batch_size, epochs = 30, validation_data =(X_test, y_test), callbacks = [ckpt])
model.save_weights('102epochs.h5')

Epoch 1/30
391/390 [==============================] - ETA: 0s - loss: 0.3832 - accuracy: 0.9088
Epoch 1: val_accuracy improved from 0.86310 to 0.88320, saving model to model.hdf5
390/390 [==============================] - 98s 251ms/step - loss: 0.3832 - accuracy: 0.9088 - val_loss: 0.4520 - val_accuracy: 0.8832
Epoch 2/30
391/390 [==============================] - ETA: 0s - loss: 0.3392 - accuracy: 0.9147
Epoch 2: val_accuracy did not improve from 0.88320
390/390 [==============================] - 98s 250ms/step - loss: 0.3392 - accuracy: 0.9147 - val_loss: 0.4460 - val_accuracy: 0.8830
Epoch 3/30
391/390 [==============================] - ETA: 0s - loss: 0.3346 - accuracy: 0.9182
Epoch 3: val_accuracy improved from 0.88320 to 0.89380, saving model to model.hdf5
390/390 [==============================] - 98s 251ms/step - loss: 0.3346 - accuracy: 0.9182 - val_loss: 0.4114 - val_accuracy: 0.8938
Epoch 4/30
391/390 [==============================] - ETA: 0s - loss: 0.3298 - accuracy: 0.91

In [10]:
model.load_weights('102epochs.h5')
from keras.preprocessing.image import ImageDataGenerator
datagen = ImageDataGenerator(rotation_range = 15, horizontal_flip = True, width_shift_range = 0.1, height_shift_range = 0.1, zoom_range = 0.2, shear_range = 15)
datagen.fit(X_train)
keras.backend.set_value(model.optimizer.lr, .001)
model.fit(datagen.flow(X_train, y_train, batch_size), steps_per_epoch = X_train.shape[0]/batch_size, epochs = 10, validation_data =(X_test, y_test), callbacks = [ckpt])

Epoch 1/10
391/390 [==============================] - ETA: 0s - loss: 0.3303 - accuracy: 0.9191
Epoch 1: val_accuracy improved from -inf to 0.87360, saving model to model.hdf5
390/390 [==============================] - 122s 262ms/step - loss: 0.3303 - accuracy: 0.9191 - val_loss: 0.5085 - val_accuracy: 0.8736
Epoch 2/10
391/390 [==============================] - ETA: 0s - loss: 0.3286 - accuracy: 0.9195
Epoch 2: val_accuracy improved from 0.87360 to 0.88990, saving model to model.hdf5
390/390 [==============================] - 97s 248ms/step - loss: 0.3286 - accuracy: 0.9195 - val_loss: 0.4439 - val_accuracy: 0.8899
Epoch 3/10
391/390 [==============================] - ETA: 0s - loss: 0.3282 - accuracy: 0.9206
Epoch 3: val_accuracy did not improve from 0.88990
390/390 [==============================] - 97s 247ms/step - loss: 0.3282 - accuracy: 0.9206 - val_loss: 0.4681 - val_accuracy: 0.8822
Epoch 4/10
391/390 [==============================] - ETA: 0s - loss: 0.3204 - accuracy: 0.9211

In [11]:
datagen = ImageDataGenerator(rotation_range = 5, horizontal_flip = True, width_shift_range = 0.05, height_shift_range = 0.05, shear_range = 5)
datagen.fit(X_train)
keras.backend.set_value(model.optimizer.lr, .001)
model.fit(datagen.flow(X_train, y_train, batch_size), steps_per_epoch = X_train.shape[0]/batch_size, epochs = 10, validation_data =(X_test, y_test), callbacks = [ckpt])

Epoch 1/10
391/390 [==============================] - ETA: 0s - loss: 0.2152 - accuracy: 0.9535
Epoch 1: val_accuracy did not improve from 0.89120
390/390 [==============================] - 97s 248ms/step - loss: 0.2152 - accuracy: 0.9535 - val_loss: 0.4339 - val_accuracy: 0.8849
Epoch 2/10
391/390 [==============================] - ETA: 0s - loss: 0.2176 - accuracy: 0.9536
Epoch 2: val_accuracy improved from 0.89120 to 0.89700, saving model to model.hdf5
390/390 [==============================] - 97s 248ms/step - loss: 0.2176 - accuracy: 0.9536 - val_loss: 0.3996 - val_accuracy: 0.8970
Epoch 3/10
391/390 [==============================] - ETA: 0s - loss: 0.2201 - accuracy: 0.9523
Epoch 3: val_accuracy did not improve from 0.89700
390/390 [==============================] - 96s 247ms/step - loss: 0.2201 - accuracy: 0.9523 - val_loss: 0.4434 - val_accuracy: 0.8856
Epoch 4/10
391/390 [==============================] - ETA: 0s - loss: 0.2222 - accuracy: 0.9528
Epoch 4: val_accuracy did not

In [12]:
datagen = ImageDataGenerator(rotation_range = 5, horizontal_flip = True, width_shift_range = 0.05, height_shift_range = 0.05, shear_range = 5)
datagen.fit(X_train)
keras.backend.set_value(model.optimizer.lr, .0001)
model.fit(datagen.flow(X_train, y_train, batch_size), steps_per_epoch = X_train.shape[0]/batch_size, epochs = 10, validation_data =(X_test, y_test), callbacks = [ckpt])

Epoch 1/10
391/390 [==============================] - ETA: 0s - loss: 0.1701 - accuracy: 0.9608
Epoch 1: val_accuracy improved from 0.89700 to 0.89750, saving model to model.hdf5
390/390 [==============================] - 97s 249ms/step - loss: 0.1701 - accuracy: 0.9608 - val_loss: 0.3613 - val_accuracy: 0.8975
Epoch 2/10
391/390 [==============================] - ETA: 0s - loss: 0.1531 - accuracy: 0.9621
Epoch 2: val_accuracy improved from 0.89750 to 0.90080, saving model to model.hdf5
390/390 [==============================] - 97s 248ms/step - loss: 0.1531 - accuracy: 0.9621 - val_loss: 0.3533 - val_accuracy: 0.9008
Epoch 3/10
391/390 [==============================] - ETA: 0s - loss: 0.1496 - accuracy: 0.9626
Epoch 3: val_accuracy did not improve from 0.90080
390/390 [==============================] - 96s 247ms/step - loss: 0.1496 - accuracy: 0.9626 - val_loss: 0.3548 - val_accuracy: 0.9005
Epoch 4/10
391/390 [==============================] - ETA: 0s - loss: 0.1477 - accuracy: 0.96

In [19]:
keras.backend.set_value(model.optimizer.lr, 0.0005)
keras.backend.set_value(model.optimizer.momentum, 0.5)
bacth_size = 512
datagen = ImageDataGenerator(rotation_range = 5, horizontal_flip = True, width_shift_range = 0.05, height_shift_range = 0.05, shear_range = 5)
datagen.fit(X_train)
model.fit(datagen.flow(X_train, y_train, batch_size), steps_per_epoch = len(X_train)//batch_size, epochs = 5, validation_data =(X_test, y_test), callbacks = [ckpt])

Epoch 1/5
390/390 [==============================] - ETA: 0s - loss: 0.1651 - accuracy: 0.9592
Epoch 1: val_accuracy did not improve from 0.90420
390/390 [==============================] - 97s 248ms/step - loss: 0.1651 - accuracy: 0.9592 - val_loss: 0.4021 - val_accuracy: 0.8909
Epoch 2/5
390/390 [==============================] - ETA: 0s - loss: 0.1680 - accuracy: 0.9599
Epoch 2: val_accuracy did not improve from 0.90420
390/390 [==============================] - 97s 248ms/step - loss: 0.1680 - accuracy: 0.9599 - val_loss: 0.4037 - val_accuracy: 0.8901
Epoch 3/5
390/390 [==============================] - ETA: 0s - loss: 0.1659 - accuracy: 0.9610
Epoch 3: val_accuracy did not improve from 0.90420
390/390 [==============================] - 97s 247ms/step - loss: 0.1659 - accuracy: 0.9610 - val_loss: 0.3762 - val_accuracy: 0.8974
Epoch 4/5
390/390 [==============================] - ETA: 0s - loss: 0.1683 - accuracy: 0.9590
Epoch 4: val_accuracy did not improve from 0.90420
390/390 [=====

In [18]:
model.evaluate(X_test,y_test)

313/313 [==============================] - 6s 18ms/step - loss: 0.3523 - accuracy: 0.9009


[0.35229936242103577, 0.9009000062942505]

The validation accuracy has been improved to 90.